In [1]:
import torch
print("CUDA dostępne:", torch.cuda.is_available())
print("Ilość GPU:", torch.cuda.device_count())
print("Nazwa GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Brak")


: 

In [1]:
import os, glob, random, math
import numpy as np
import nibabel as nib
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt

from transformers import (
    AutoImageProcessor,
    SegformerForSemanticSegmentation,
    get_cosine_schedule_with_warmup,
)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)



: 

In [3]:
import sys
print(sys.executable)  # sprawdź, że to ...\Miniconda3\envs\segformer_env\python.exe


c:\Users\igorp\miniconda3\envs\segformer_env\python.exe
